In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

# Excelファイルを指定のシートに読み込む
wb = load_workbook('example_with_formula.xlsx')
ws = wb['Sheet1']

# 追加したい計算式と列名を指定
add_formulas = {"計算1": "後品番*(先品番+先品番)", "計算2": "先品番*3"}

# 読み込んだシートをデータフレームに変換
data = [[cell.value for cell in row] for row in ws.iter_rows()]
df = pd.DataFrame(data[1:], columns=data[0])

# 列名とセルの列の辞書を作成
column_letters = {name: get_column_letter(idx + 1) for idx, name in enumerate(df.columns)}

# 読み込んだ列名と数式を対応させる
for column_name, formula in add_formulas.items():
    for column in df.columns:
        formula = formula.replace(column, column_letters[column] + "{row}")
    add_formulas[column_name] = formula

# 列名と数式の追加
for column_name, formula in add_formulas.items():
    # 列名を追加する列を指定
    next_column_number = ws.max_column + 1
    next_column_letter = get_column_letter(next_column_number)

    # 列名を追加
    ws[f'{next_column_letter}1'] = column_name

    # 各セルに数式を適用
    for row in range(2, len(df) + 2):  # openpyxl is 1-indexed
        ws[f'{next_column_letter}{row}'] = f'={formula.format(row=row)}'

# Save the workbook
wb.save('example_with_formula.xlsx')

In [ ]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

def prepare_formulas(headers, formulas):
    """
    Prepare new formulas based on the column headers and the original formulas.
    This function does not modify the original formulas.

    Parameters:
    headers (list): List of column headers.
    formulas (dict): Dictionary of original formulas.

    Returns:
    dict: New formulas.
    """
    # Assign column letter to each header.
    column_letters = {name: get_column_letter(idx) for idx, name in enumerate(headers, start=1)}

    new_formulas = {}
    for column_name, formula in formulas.items():
        new_formula = formula
        # Replace column name with the cell reference in the formula.
        for column in headers:
            new_formula = new_formula.replace(column, column_letters[column] + "{row}")
        new_formulas[column_name] = new_formula

    return new_formulas

def apply_formulas(ws, formulas):
    """
    Apply the formulas to the Excel worksheet.

    Parameters:
    ws (Worksheet): The Excel worksheet.
    formulas (dict): Formulas to be applied.
    """
    # Create new columns and apply the formula for each row.
    for column_name, formula in formulas.items():
        next_column_number = ws.max_column + 1
        next_column_letter = get_column_letter(next_column_number)
        ws[f'{next_column_letter}1'] = column_name
        for row in range(2, ws.max_row + 1):
            ws[f'{next_column_letter}{row}'] = f'={formula.format(row=row)}'

# Load an Excel workbook.
wb = load_workbook('example_with_formula.xlsx')
ws = wb['Sheet1']

# Formulas to be applied.
add_formulas = {"計算1": "後品番*(先品番+先品番)", "計算2": "先品番*3"}

# Extract column headers from the first row of the worksheet.
headers = [cell.value for cell in ws[1]]

# Prepare new formulas.
new_formulas = prepare_formulas(headers, add_formulas)

# Apply the new formulas to the worksheet.
apply_formulas(ws, new_formulas)

# Save the workbook with a new name.
wb.save('example_with_formula_updated.xlsx')

In [ ]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from collections import OrderedDict

def prepare_formulas(headers, formulas):
    """
    ヘッダーと元の計算式に基づいてセル参照の計算式を作成します。
    この関数は元の計算式を変更しません。

    引数:
    headers (list): 列ヘッダーのリスト。
    formulas (dict): 元の計算式の辞書。

    戻り値:
    dict: セル参照の計算式。
    """
    # 各ヘッダーに列番号を割り当てます。
    column_letters = {name: get_column_letter(idx) for idx, name in enumerate(headers, start=1)}

    new_formulas = {}
    for column_name, formula in formulas.items():
        new_formula = formula
        # 計算式中の列名をセル参照に置換します。
        for column in column_letters.keys():
            new_formula = new_formula.replace(column, column_letters[column] + "{row}")
        new_formulas[column_name] = new_formula
        # 追加された列名とその位置も記録します。
        column_letters[column_name] = get_column_letter(len(column_letters) + 1)

    return new_formulas

def apply_formulas(ws, formulas):
    """
    計算式をExcelワークシートに適用します。

    引数:
    ws (Worksheet): Excelのワークシート。
    formulas (dict): 適用する計算式。例：{"add_column": 'A{row}+B{row}'}
    """
    # 新しい列を作成し、各行に計算式を適用します。
    for column_name, formula in formulas.items():
        next_column_number = ws.max_column + 1
        next_column_letter = get_column_letter(next_column_number)
        ws[f'{next_column_letter}1'] = column_name
        for row in range(2, ws.max_row + 1):
            ws[f'{next_column_letter}{row}'] = f'={formula.format(row=row)}'

# Excelのワークブックを読み込みます。
wb = load_workbook('example_with_formula.xlsx')
ws = wb['Sheet1']

# 追加する計算式。(excelの計算式)
add_formulas = OrderedDict({"計算1": "後品番*(後品番+先品番)",
                            "計算2": "計算1^2",
                            "計算3": "計算1*計算2",
                            })

# ワークシートの最初の行から列ヘッダーを抽出します。
headers = [cell.value for cell in ws[1]]

# 新しい計算式を準備します。
new_formulas = prepare_formulas(headers, add_formulas)

# ワークシートに新しい計算式を適用します。
apply_formulas(ws, new_formulas)

# ワークブックを新しい名前で保存します。
wb.save('example_with_formula_updated.xlsx')


In [ ]:
# これが最終

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

def prepare_formulas(headers, formulas):
    """
    ヘッダーと元の計算式に基づいてセル参照の計算式を作成します。
    この関数は元の計算式を変更しません。

    引数:
    headers (list): 列ヘッダーのリスト。
    formulas (dict): 元の計算式の辞書。

    戻り値:
    dict: セル参照の計算式。
    """
    # 各ヘッダーに列番号を割り当てます。
    column_letters = {name: get_column_letter(idx) for idx, name in enumerate(headers, start=1)}

    new_formulas = {}
    for column_name, formula in formulas.items():
        new_formula = formula
        # 計算式中の列名をセル参照に置換します。
        for column in column_letters.keys():
            new_formula = new_formula.replace(column, column_letters[column] + "{row}")
        new_formulas[column_name] = new_formula
        # 追加された列名とその位置も記録します。
        column_letters[column_name] = get_column_letter(len(column_letters) + 1)

    return new_formulas

def apply_formulas(ws, formulas):
    """
    計算式をExcelワークシートに適用します。

    引数:
    ws (Worksheet): Excelのワークシート。
    formulas (dict): 適用する計算式。例：{"add_column": 'A{row}+B{row}'}
    """
    # 新しい列を作成し、各行に計算式を適用します。
    for column_name, formula in formulas.items():
        next_column_number = ws.max_column + 1
        next_column_letter = get_column_letter(next_column_number)
        ws[f'{next_column_letter}1'] = column_name
        for row in range(2, ws.max_row + 1):
            ws[f'{next_column_letter}{row}'] = f'={formula.format(row=row)}'

# Excelのワークブックを読み込みます。
wb = load_workbook('example_with_formula.xlsx')
ws = wb['Sheet1']

# 追加する計算式。(excelの計算式)
add_formulas = {"計算1": "後品番*(後品番+先品番)",
                "計算2": "計算1^2",
                "計算4": "計算1*計算2",
                }

# ワークシートの最初の行から列ヘッダーを抽出します。
headers = [cell.value for cell in ws[1]]

# 新しい計算式を準備します。
new_formulas = prepare_formulas(headers, add_formulas)

# ワークシートに新しい計算式を適用します。
apply_formulas(ws, new_formulas)

# ワークブックを新しい名前で保存します。
wb.save('example_with_formula_updated.xlsx')